<a href="https://colab.research.google.com/github/yuj0630/Catholic_Regacy/blob/main/style_transfer_tutorial_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요한 PyTorch 라이브러리 불러오기
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models

import PIL
import matplotlib.pyplot as plt

import copy

In [ ]:
#GPU 장치 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def image_loader(img_path, imsize):
  loader = transforms.Compose([
      transforms.Resize(imsize), #이미지의 크기를 변경
      transforms.ToTensor() # torch, Tensor 형식으로 변경
  ])
  image = PIL.Image.open(img_path) 
  # 네트워크 입력에 들어갈 이미지에 배치 목적의 차원 추가
  image = loader(image).unsqueeze(0)
  return image.to(device, torch.float) # GPU로 올리기

# torch, Tensor 형태의 이미지를 화면에 출력
def imshow(tensor):
  #matplotlib는 CPU 기반이므로 CPU로 옮기기
  image = tensor.cpu().clone()
  # torch.Tensor에서 사용되는 배치 목적의 차원 제거
  image = image.squeeze(0)
  image = transforms.ToPIL()(image)
  # 이미지를 화면에 출력(matplotlib는 [0, 1] 사이의 값이라고 해도 정상적으로 처라)
  plt.imshow(image)
  plt.show()

In [ ]:
!git clone https://github.com/ndb796/Deep_Learning-Paper-Review-and-Practice
%cd Deep-Learning-Paper-Review-and-Practice

Cloning into 'Deep_Learning-Paper-Review-and-Practice'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: 'Deep-Learning-Paper-Review-and-Practice'
/content


# Image Reconstruction 실습

In [ ]:
imag_path = './code_practice/images/cat.jpg'
target_image = image_loader(img_path, (512, 512))
imshow(target_image)

NameError: ignored

In [ ]:
# 동일한 크기의 노이즈 이미지 준비하기
noise = torch.empty_like(target_image).uniform_(0, 1).to(device)
imshow(noise)

NameError: ignored

In [ ]:
loss = nn.MSELoss() # 손실(loss) 함수 설정
iters = 100 #반복 횟수 설정
lr = 1e4

print("[Start]")
imshow(noise)

for i in range(iters):
  # required_grad 속성의 값을 True로 설정하여 해당 torch, Tensor의 연산을 추적
  noise.requires_grad = True

  #손실 함수에 대하여 미분하여 기울기 계산
  output = loss(noise, target_image)
  output.backward()

  #계산된 기울기를 이용하여 손실함수가 감소하는 방향으로 업데이트
  gradient = lr * noise.grad
  # 결과적으로 노이즈의 각 픽셀의 값이 [-eps, eps] 사이의 값이 되도록 자르기
  noise = torch.clamp(noise = gradient, min=0, max=1).detach_()

  if (i + 1) % 10 == 0:
    print(f'[Step: {i + 1}]')
    print(f"Loss: {output}")
    imshow(noise)

# 실습을 위한 이미지 불러오기

In [ ]:
# 콘텐츠 이미지와 스타일 이미지를 모두 준비합니다.
content_img = image_loader('./code_practices/images/content_img_1.jpg', (512, 640))
style_img = image_loader('./code_practices/images/style_img_1.jpg', (512, 640))

print("[Content Image]")
imshow(content_img)
print("[Style Image]")
imshow(style_img)

# CNN 네트워크 불러오기

In [ ]:
# 뉴럴 네트워크 모델을 불러옵니다.
cnn = models.vgg19(pretained=True),features.to(device).eval()
print(cnn)

TypeError: ignored

In [ ]:
# 입력 정규화를 위한 초기화
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

class Normalization(nn.Module):
  def __init__(self, mean, std):
    super(Normalization, self).__init__()
    self.mean = mean.clone().view(-1, 1, 1)
    self.std = std.clone().view(-1, 1, 1)

  def forward(self, img):
    return (img - self.mean) / self.std  

# Style Reconstruction 실습하기



*   임의의 노이즈가 측정한 이미지의 스타일을 가지도록 변환합니다.
*   style_layers 리스트 변수의 값을 조절하여 어떤 레이어를 이용할지 설정할 수 있습니다.



In [ ]:
def gram_matrix(input):
  a, b, c, d = input.size()
  features = input.view(a * b, c * d)
  G = torch.mm(features, features.t())
  return G.div(a * b * c * d)

# 스타일 손실 계산을 위한 클래스 정의
class StyleLoss(nn.Module):
  def __init__(self, target_feature):
    super(StyleLoss, self).__init__()
    self.target = gram_matrix(target_feature).detach()

  def forward(self, input):
    G = gram_matrix(input)
    self.loss = F.mse_loss(G, self.target)
    return input     

In [ ]:
style_layers = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

# 스타일 손실을 계산하는 함수
def get_style_losses(cnn, style_img, noise_image